Here is my notebook of a sample chess game using a 8x3 board. Each player is given 6 pieces across the left end/strip of the board.
We will set the computer to the first player(white).
The minmax recursive function will use a basic evalutation function, adding to the computers score for its pieces, and subtracting for blacks.

In [10]:
pip install python-chess

In [11]:
import chess
import numpy as np
import random
import functools

In [12]:
# Create an empty board
board = chess.Board(None)  # no pieces

# Place 3 white pieces on the first row
board.set_piece_at(chess.A2, chess.Piece(chess.PAWN, chess.WHITE))
board.set_piece_at(chess.B2, chess.Piece(chess.PAWN, chess.WHITE))
board.set_piece_at(chess.C2, chess.Piece(chess.PAWN, chess.WHITE))

# Place 3 black pieces on the first row
board.set_piece_at(chess.A7, chess.Piece(chess.PAWN, chess.BLACK))
board.set_piece_at(chess.B7, chess.Piece(chess.PAWN, chess.BLACK))
board.set_piece_at(chess.C7, chess.Piece(chess.PAWN, chess.BLACK))


# Three offesnsive pieces white
board.set_piece_at(chess.A1, chess.Piece(chess.QUEEN, chess.WHITE))
board.set_piece_at(chess.B1, chess.Piece(chess.KING, chess.WHITE))
board.set_piece_at(chess.C1, chess.Piece(chess.ROOK, chess.WHITE))

# Three offesnive pieces black
board.set_piece_at(chess.A8, chess.Piece(chess.QUEEN, chess.BLACK))
board.set_piece_at(chess.B8, chess.Piece(chess.KING, chess.BLACK))
board.set_piece_at(chess.C8, chess.Piece(chess.ROOK, chess.BLACK))


In [13]:
print("8 Row 3 Column Board")
print(board)

8 Row 3 Column Board
q k r . . . . .
p p p . . . . .
. . . . . . . .
. . . . . . . .
. . . . . . . .
. . . . . . . .
P P P . . . . .
Q K R . . . . .


In [14]:
board.piece_map().items()

dict_items([(58, Piece.from_symbol('r')), (57, Piece.from_symbol('k')), (56, Piece.from_symbol('q')), (50, Piece.from_symbol('p')), (49, Piece.from_symbol('p')), (48, Piece.from_symbol('p')), (10, Piece.from_symbol('P')), (9, Piece.from_symbol('P')), (8, Piece.from_symbol('P')), (2, Piece.from_symbol('R')), (1, Piece.from_symbol('K')), (0, Piece.from_symbol('Q'))])

In [15]:
# Create min max algorithm for computer to make the best move
def evaluate_board(board):
  """
  Simple evaluation function:
  Assigns point values to pieces and sums the values for each player.
  Positive values favor White, negative values favor Black.
  """
  piece_values = {
      chess.PAWN: 1, # Least Important
      chess.ROOK: 5, # More Important
      chess.QUEEN: 12, # Very Important
      chess.KING: 0, # King value is for checkmate, not material(Not to capture)
  }
  evaluation = 0
  for square in chess.SQUARES:
    piece = board.piece_at(square)
    if piece:
      value = piece_values.get(piece.piece_type, 0)
      # Add score for white(the computer)
      if piece.color == chess.WHITE:
        evaluation += value
      # Subtract for blacks pieces
      else:
        evaluation -= value
  return evaluation

# @functools.cache
def min_max(board, depth, maximizing_player):
  # -- Base case
  if depth == 0 or board.is_game_over():
    return evaluate_board(board)

  # Max player case
  if maximizing_player:
    # loop through all possible moves to recursievely call upon
    max_eval = float('-inf')
    for move in board.legal_moves:
      board.push(move)
      eval = min_max(board, depth - 1, False)
      board.pop()
      max_eval = max(max_eval, eval)
    return max_eval

  # Min player case
  else:
    min_eval = float('inf')
    for move in board.legal_moves:
      board.push(move)
      eval = min_max(board, depth - 1, True)
      board.pop()
      min_eval = min(min_eval, eval)
    return min_eval


# Create function to actually make the move utilizing the min_max algorithm
def make_move(board, valid_squares):
  best_move = None
  best_eval = float('-inf')
  # Filter legal moves to stay within valid_squares
  possible_moves = [move for move in board.legal_moves if move.from_square in valid_squares and move.to_square in valid_squares]

  for move in possible_moves:
    board.push(move)
    eval = min_max(board, 3, False)
    board.pop()
    if eval > best_eval:
      best_eval = eval
      best_move = move
  return best_move

In [16]:
valid_squares = {chess.square(file, rank) for file in range(0, 3) for rank in range(0, 8)}
#print(valid_squares)


# --- Play loop ---
while not board.is_game_over():
    print("Current board:")
    print(board)

    if board.turn == chess.WHITE:  # AI (random move)
        legal_moves = [m for m in board.legal_moves if (m.from_square in valid_squares and m.to_square in valid_squares)]
        if not legal_moves:
            print("White has no legal moves. Game over.")
            break

        # Implement minmax for the computer here
        move = make_move(board, valid_squares)
        #move = random.choice(legal_moves)
        print(f"White plays: {move.uci()}")
        print("---------------------------------------------------------\n")
        board.push(move)

    else:  # Human move
        legal_moves = [m for m in board.legal_moves if (m.from_square in valid_squares and m.to_square in valid_squares)]
        #legal_moves_NoQuit = [m for m in board.legal_moves if (m.from_square in valid_squares and m.to_square in valid_squares)]
        if not legal_moves:
            print("Black has no legal moves. Game over.")
            break

        print("Your turn! Enter a move in UCI format (e.g., a7a6, b7b6, c7c6)")
        print("Legal moves:", [m.uci() for m in legal_moves])


        #game_over = False
        while True:
            try:
                user_input = input("Your move: ")
                move = chess.Move.from_uci(user_input)

                # -- Create a case for quiting the game completely
                if user_input.lower() == "quit":
                    game_over = True
                    print("Quitting the game.")
                    break

                if move in legal_moves:
                    break
            except ValueError:
                print("Invalid move format. Try again.")
                print("Legal moves:", [m.uci() for m in legal_moves])



        print("---------------------------------------------------------\n")

        if move in legal_moves:
            board.push(move)
        else:
            print("Invalid move, try again.")
            continue

print("\nFinal board:")
print(board)
print("Game result:", board.result())

Current board:
q k r . . . . .
p p p . . . . .
. . . . . . . .
. . . . . . . .
. . . . . . . .
. . . . . . . .
P P P . . . . .
Q K R . . . . .
White plays: c2c3
---------------------------------------------------------

Current board:
q k r . . . . .
p p p . . . . .
. . . . . . . .
. . . . . . . .
. . . . . . . .
. . P . . . . .
P P . . . . . .
Q K R . . . . .
Your turn! Enter a move in UCI format (e.g., a7a6, b7b6, c7c6)
Legal moves: ['c7c6', 'b7b6', 'a7a6', 'c7c5', 'b7b5', 'a7a5']


KeyboardInterrupt: Interrupted by user